
# **6. Pytorch를 이용한 MNIST classifier 구현**

목표 : 라이브러리 pytorch를 이용하여 MNIST 데이터를 분류하는 분류기를 구현해봅시다.
---

normalization 전처리를 주석처리했습니다.
이 부분을 풀고 학습해보세요.




In [ ]:
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

def one_hot_embedding(labels, num_classes):
    # 단일 라벨 텐서를 원핫 벡터로 바꿔줍니다.
    y = torch.eye(num_classes)
    one_hot = y[labels]
    return one_hot

def softmax_to_one_hot(tensor):
    # softmax 결과를 가장 높은 값이 1이 되도록 하여 원핫 벡터로 바꿔줍니다. acuuracy 구할 때 씁니다.
    max_idx = torch.argmax(tensor, 1, keepdim=True)
    if tensor.is_cuda :
        one_hot = torch.zeros(tensor.shape).cuda()
    else:
        one_hot = torch.zeros(tensor.shape)
    one_hot.scatter_(1, max_idx, 1)
    return one_hot

def weight_init(m):
    classname = m.__class__.__name__
    # m에서 classname이 Linear(신경망 레이어)인 경우
    if classname.find('Linear') != -1:
        # weight를 uniform distribution을 이용하여 초기화하고 bias는 0으로 초기화
        m.weight.data.uniform_(0.0, 1.0)
        m.bias.data.fill_(0)


class TwoLayerNet_pytorch(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.network1 = nn.Sequential(
        nn.Linear(self.input_size, self.hidden_size),
        nn.Sigmoid(),
        nn.Linear(self.hidden_size, self.output_size),
        nn.Softmax()
    )
  def forward(self, x):
    y = self.network1(x)
    return y

epochs = 5
learning_rate = 0.01
batch_size = 100
loss_function = nn.BCELoss()

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                       #,transforms.Normalize((0.1307,), (0.3081,)) # 한번 돌려본 후, 돌아가는것을 확인했다면 이 주석을 지우세요.
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                       #,transforms.Normalize((0.1307,), (0.3081,)) # 한번 돌려본 후, 돌아가는것을 확인했다면 이 주석을 지우세요.
                   ])),
    batch_size=batch_size, shuffle=True)



net = TwoLayerNet_pytorch(input_size=784, hidden_size=50, output_size=10)
net.apply(weight_init)

optimizer = optim.SGD(net.parameters(), lr=learning_rate)

train_loss_list = [] # 결과 출력을 위한 코드
net.train() # 학습할것을 명시하여 자원낭비를 줄이는 코드
for epoch in range(epochs):
    for i, (X, t) in enumerate(train_loader):
        X = X.view(-1, 784) # 1 x 28 x 28 형태임으로, 784 형태의 벡터로 바꿔준다.
        t = one_hot_embedding(t, 10) # 숫자로 출력됨으로 원핫코드로 바꿔준다.

        # 순전파
        Y = net(X)
        loss = loss_function(Y, t)

        train_loss_list.append(loss) # 결과 출력을 위한 코드
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("[%d/%d][%d/%d] loss : %f"%(i,len(train_loader),epoch,epochs, loss))

print("calculating accuracy...")
net.eval() # 학습하지 않을 것을 명시하여 자원낭비를 줄이는 코드

correct = 0
with torch.no_grad():
    for i, (X, t) in enumerate(test_loader):
        X = X.view(-1, 784)
        t = one_hot_embedding(t, 10)
        Y = net(X)

        onehot_y= softmax_to_one_hot(Y) 
        correct += int(torch.sum(onehot_y * t)) # testset에서 정답을 맞춘 횟수 저장
print("Accuracy : %f" % (100. * correct / len(test_loader.dataset)))
plt.plot(train_loss_list)